# **Feature Extraction Methods: Imbalanced Data With Annotations**

- *Key Features*: [MFCCs, Mel-Spectrograms, Chroma Frequencies, RMS Power]
- *Key Manipulations*: [Varying Window Sizes, Normalization, Average Pooling (Compression), Filtering]
- *Process Assistence*: [Converting them to numpy arrays now, easy label access across features]
- *Conversion*: [To numpy arrays and pkl files]


In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import os
import time

# Libraries for audio
from IPython.display import Audio
import librosa
import librosa.display

# Training and Testing Split
from sklearn.model_selection import train_test_split

# for normalization & avgpooling features
import tensorflow as tf
# for preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
import random
import IPython.display as ipd

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Variabels to be reused
path = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/audio_files' 
npy_path = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/train_audio_npy/' 
train_csv = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/train-not-annotated.csv' 
annotated_train_csv = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/trainval-split/trainval-annotated.csv'
not_annotated_splt = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/trainval-split/trainval.csv'
sr = 22050

In [3]:
trainval_data = pd.read_csv(annotated_train_csv)
train_data = trainval_data[trainval_data['set'] == 'tr']
val_data = trainval_data[trainval_data['set'] == 'val']

# **Creating a class to do the extraction**

In [4]:
class Extraction:
  def __init__(self, train_df, val_df, window_size, overlap=0.5, npy_path=npy_path, sr=sr, n_mels=128, n_mfcc=20, n_chroma=12, n_cqt=84, hoplength=256, features=['mfcc'], normalize=True, avgpool=False):
    """
    Instantiate the Extraction class to extract features.

    Parameters:
      sr (int): Sample rate of the audio files.
      n_mfccs (int): Number of MFCCs to extract.
      n_mels (int): Number of Mel bands to extract.
      n_chroma (int): Number of chroma bins to use.
      n_cqt (int): Number of CQT bins to use.
      features (list): List of features to extract.
        accepted features: 'mfcc', 'chroma', 'cqt', 'melspectrogram'.
      normalize (bool): Whether to normalize the features.
      avgpool (bool): Whether to avgpool the features.
    """
    
    self.train_df = train_df
    self.val_df = val_df
    self.npy_path = npy_path
    self.window_size = window_size
    self.overlap = overlap
    self.sr = sr
    self.n_mels = n_mels
    self.n_mfcc = n_mfcc
    self.n_chroma = n_chroma
    self.n_cqt = n_cqt
    self.hoplength = hoplength

     # confirm features have been specified
    assert len(features) != 0, "Must Specify At Least One Feature In The Form Of A List."
    self.features = features

    self.accepted_feature = ['mfcc', 'chroma', 'cqt', 'melspectrogram']
    for feature in self.features:
      assert feature in self.accepted_feature, f"{feature} is not an accepted feature, only 'mfcc', 'chroma', 'cqt', 'melspectrogram' are accepted features."

    self.normalize = normalize
    self.avgpool = avgpool

    print(f"Train DataFrame shape: {train_df.shape}")
    print(f"Validation DataFrame shape: {val_df.shape}")

    # extract train and val labels and features
    self.train_y, self.train_features, self.train_ids = self.feature_extraction(self.train_df, window_size=self.window_size)
    self.val_y, self.val_features, self.val_ids = self.feature_extraction(self.val_df, window_size=self.window_size)

    # process the features by average pooling
    self.train_features_2D, self.val_features_2D, self.train_features_1D, self.val_features_1D = self.process_features(self.train_features, self.val_features)


  def normalize_audio(self, audio):
    return (audio - np.min(audio)) / (np.max(audio) - np.min(audio))
  
  def generate_pink_noise(self, num_samples):
    white_noise = np.random.randn(num_samples)
    
    # Apply a filter to convert white noise into pink noise (1/f noise)
    X = np.fft.rfft(white_noise)
    S = np.arange(1, len(X) + 1)  # Frequency scaling
    pink_noise = np.fft.irfft(X / S)

    if len(pink_noise) < num_samples:
        # Pad with zeros if the length is less than num_samples
        pink_noise = np.pad(pink_noise, (0, num_samples - len(pink_noise)), mode='constant')
    elif len(pink_noise) > num_samples:
        # Trim if necessary
        pink_noise = pink_noise[:num_samples]
    
    return self.normalize_audio(pink_noise)
  
  def pad_with_noise(self, audio_data, window_length, window_samples):
    current_length = librosa.get_duration(y=audio_data, sr=self.sr)

    if current_length > window_length:
        return audio_data
    
    target_length_samples = int(window_length * sr) 
    current_length_samples = window_samples
    padding_length_samples = target_length_samples - current_length_samples

    assert target_length_samples == (current_length_samples+padding_length_samples)
    
    # Generate pink noise to pad with
    pink_noise = self.generate_pink_noise(padding_length_samples)
    padded_audio = np.concatenate([audio_data, pink_noise])
    # if len(padded_audio) < target_length_samples:
    #     padded_audio = np.append(padded_audio, self.generate_pink_noise(1))

    assert target_length_samples == len(padded_audio)
    
    return padded_audio
  
  def avg_pooling_keras(self, feature):
    # Clear the previous Keras session
    tf.keras.backend.clear_session()

    # Define the input shape based on features
    input_shape = feature.shape[1:]  # (n_mels, time_steps)

    # Create the Keras model for average pooling
    inputs = tf.keras.layers.Input(shape=input_shape)
    pooled = tf.keras.layers.GlobalAveragePooling1D()(inputs)
    pooling_model = tf.keras.models.Model(inputs=inputs, outputs=pooled)

    # Perform pooling using the model
    pooled_features = pooling_model.predict(feature)

    return pooled_features

#-------------------------Feature Extraction---------------------------------------
  def extract_mfcc(self, window):
    mfcc = librosa.feature.mfcc(y=window, sr=self.sr, n_mfcc=self.n_mfcc, hop_length=self.hoplength)
    if self.normalize:
      return librosa.util.normalize(mfcc)
    else:
      return mfcc


  def extract_chroma(self, window):
    chroma = librosa.feature.chroma_stft(y=window, sr=self.sr, n_chroma=self.n_chroma, hop_length=self.hoplength)
    if self.normalize:
      return librosa.util.normalize(chroma)
    else:
      return chroma
   

  def extract_cqt(self, window):
    cqt = librosa.cqt(y=window, sr=sr, hop_length=self.hoplength, n_bins=self.n_cqt)
    cqt_db = librosa.amplitude_to_db(np.abs(cqt), ref=np.max)
    return cqt_db

  def extract_melspectrogram(self, window):
    mel = librosa.feature.melspectrogram(y=window, sr=self.sr, n_mels=self.n_mels, hop_length=self.hoplength)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    if self.normalize:
      return librosa.util.normalize(mel_db)
    else:
      return mel_db
    
  def avgpooling(self, train_X, val_X, n_time, n_features):
    """
    Average pooling the train and val features.

    Parameters:
      train_X (npy): Training feature array of shape (batch_size, n_features, n_time)
      val_X (npy): Validation feature array of shape (batch_size, n_features, n_time)
      n_time (int): Time axis
      n_features (int): Feature axis

    Returns:
      train_X (npy): Avgpooled training feature array of shape (batch_size, n_features)
      val_X (npy): Avgpooled validation feature array of shape (batch_size, n_features)
    """
    # Clear the Keras session
    tf.keras.backend.clear_session()
    
    # Create the Keras input layer with shape (n_features, n_time)
    input_layer = tf.keras.layers.Input(shape=(n_features, n_time))
    
    # Apply average pooling over the time axis (axis=-1) to reduce n_time
    avg_pool = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1))(input_layer)
    
    # Build the model
    pooling_model = tf.keras.models.Model(inputs=input_layer, outputs=avg_pool)

    # Use the model to apply average pooling on the training and validation features
    train_X = pooling_model.predict(train_X)
    val_X = pooling_model.predict(val_X)

    return train_X, val_X

    
  def process_features(self, train_features_dict, val_features_dict):
    train_copy = train_features_dict.copy()
    val_copy = val_features_dict.copy()
    for each in train_copy.keys():
      
      if each == 'mfcc':
        n_features=self.n_mfcc
      elif each == 'chroma':
        n_features=self.n_chroma
      elif each == 'cqt':
        n_features=self.n_cqt
      elif each == 'melspectrogram':
        n_features=self.n_mels
      
      train_feature = train_copy[each]
      val_feature = val_copy[each]

      if self.avgpool:
        train_copy[each], val_copy[each] = self.avgpooling(train_feature, val_feature, n_time=train_feature.shape[2], n_features=n_features)
      else:
        train_copy[each], val_copy[each] = train_copy[each], val_copy[each]
    
    return train_features_dict, val_features_dict, train_copy, val_copy
      

  def feature_extraction(self, dataframe, window_size):
    y = [] # To hold the labels
    ids = []
    features_dict = {item: [] for item in self.features} # Create a key for each feature listed
    print(f"Number of rows in dataframe: {len(dataframe)}")
    for _, row in tqdm(dataframe.iterrows(), desc="Processing data", total=len(dataframe)):
          label = row['species']
          file_path = os.path.join(self.npy_path, row['filename_npy'])
          id = row['audio_name']
          start = row['start']
          end = row['end']

          # print(f"Processing file: {file_path}")

          try:
            
              audio = np.load(file_path)
          except FileNotFoundError:
              print(f"File not found: {file_path}")
              continue


          start = int(start * sr)
          end = int(end * sr)+512



          if end > len(audio):
             end = len(audio)

          sample = audio[start:end]

          if len(sample) < 512:
                continue

          sample = self.normalize_audio(sample)

          sample = self.pad_with_noise(sample, window_length=self.window_size, window_samples=len(sample))
          # print(len(sample))

          window_samples = int(window_size * self.sr)
          hop_samples = int(window_samples * (1 - self.overlap))  # For overlapping

          # Break the audio into windows with the specified overlap
          audio_windows = librosa.util.frame(sample, frame_length=window_samples, hop_length=hop_samples).T
          
          
          # display(label)
          
          for _, window in enumerate(audio_windows):
              
              y.append(label)
              ids.append(id)

              if len(window) < window_samples:
                  if len(window) < 512*2:
                     continue
                  else:
                      window = self.pad_with_noise(window, window_length=window_size)
              
              # Feature Extraction FR --------------------------------------------------------------------
              # dynatically call the extract_x function to extract the listed features
              for feature in self.features:
                extract = f"extract_{feature}"
                if hasattr(self, extract) and callable(func := getattr(self, extract)):
                  features_dict[feature].append(func(window))

          # cast lists to np arrays
    for each in features_dict.keys():
              features_dict[each] = np.array(features_dict[each])

    y = np.array(y)
    ids = np.array(ids)

          # If not using average pooling, return resized features
    return y, features_dict

## **Window Size = 3s**

In [ ]:
features_list = ['melspectrogram', 'mfcc', 'chroma', 'cqt']

In [ ]:
features = Extraction(train_data,
                      val_data,
                      window_size=3,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [08:21<00:00,  1.03s/it]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [01:41<00:00,  1.28it/s]


708/708 ━━━━━━━━━━━━━━━━━━━━ 0s 471us/step
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 437us/step
708/708 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step
708/708 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 397us/step
708/708 ━━━━━━━━━━━━━━━━━━━━ 0s 393us/step
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 387us/step


In [ ]:
train_y = features.train_y
val_y = features.val_y

display(train_y.shape)
display(val_y.shape)

(10981,)

'melspectrogram'

(10981, 60)

'mfcc'

(10981, 20)

'chroma'

(10981, 12)

'rms'

(10981, 1)

In [ ]:
# Avgpooled Features
train_features_1D = features.train_features_1D
for key in train_features_1D.keys():
  display(key)
  display(train_features_1D[key].shape)

In [ ]:
# Avgpooled Features
val_features_1D = features.val_features_1D
for key in val_features_1D.keys():
  display(key)
  display(val_features_1D[key].shape)

In [ ]:
# Not avgpooled Features
train_features_2D = features.train_features_2D
for key in train_features_2D.keys():
  display(key)
  display(train_features_2D[key].shape)

In [ ]:
# Not avgpooled Features
val_features_2D = features.val_features_2D
for key in val_features_2D.keys():
  display(key)
  display(val_features_2D[key].shape)

In [ ]:
train_ids = features.train_ids
val_ids = features.val_ids

### Encode Classes

In [ ]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [ ]:
# Avg Pooled

train_features_1D['label'] = train_y_encoded
val_features_1D['label'] = val_y_encoded

train_features_1D['id'] = train_ids
val_features_1D['id'] = val_ids

# Not Avg Pooled

train_features_2D['label'] = train_y_encoded
val_features_2D['label'] = val_y_encoded

train_features_2D['id'] = train_ids
val_features_2D['id'] = val_ids

In [ ]:
merged_dict_1D = {'train': train_features_1D, 'val': val_features_1D}
merged_dict_1D

In [ ]:
merged_dict_2D = {'train': train_features_2D, 'val': val_features_2D}
merged_dict_2D

{'train': {'melspectrogram': array([[1.7294491e-05, 1.8239583e-05, 1.9485902e-05, ..., 5.1940111e-07,
          3.2287385e-08, 2.7940120e-08],
         [7.9314253e-07, 8.0714307e-07, 8.3096978e-07, ..., 1.3027967e-06,
          8.4759665e-07, 8.0172282e-07],
         [3.0612518e-06, 3.0607123e-06, 3.0592107e-06, ..., 6.0072955e-07,
          3.6999687e-08, 1.4126979e-08],
         ...,
         [1.0815166e-06, 1.0910663e-06, 1.1071655e-06, ..., 4.2367597e-07,
          1.3173792e-07, 1.2475495e-07],
         [2.1907644e-07, 2.7223422e-07, 3.6665881e-07, ..., 5.7446897e-07,
          1.8625977e-07, 1.7997159e-07],
         [2.4198832e-06, 2.6966893e-06, 3.1864115e-06, ..., 1.2659899e-05,
          2.3157411e-07, 2.2612821e-07]], dtype=float32),
  'mfcc': array([[-1.0000000e+00, -2.3626368e-01, -5.0428110e-01, ...,
           5.1737577e-03,  1.1148715e-02,  3.4610374e-04],
         [-1.0000000e+00, -2.4848318e-01, -4.7058651e-01, ...,
           2.9826174e-03,  9.7121680e-03, -5.3395964e

### Save the merged dictionary to a pkl

In [ ]:
# Avrg Pooled
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/Annotated/Regular/AveragePooled/split_features_3s_all_1D.pkl', 'wb') as file:
  pickle.dump(merged_dict_1D, file)
del file

In [ ]:
# Avrg Pooled
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/Annotated/Regular/NotAveragePooled/split_features_3s_all_2D.pkl', 'wb') as file:
  pickle.dump(merged_dict_2D, file)
del file

## **Window Size = 1s**

In [ ]:
features_list = ['melspectrogram', 'mfcc', 'chroma', 'cqt']

In [ ]:
features = Extraction(train_data,
                      val_data,
                      window_size=1,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (485, 6)
Validation DataFrame shape: (130, 6)
Number of rows in dataframe: 485


Processing data: 100%|██████████| 485/485 [08:21<00:00,  1.03s/it]


Number of rows in dataframe: 130


Processing data: 100%|██████████| 130/130 [01:41<00:00,  1.28it/s]


708/708 ━━━━━━━━━━━━━━━━━━━━ 0s 471us/step
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 437us/step
708/708 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step
708/708 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 397us/step
708/708 ━━━━━━━━━━━━━━━━━━━━ 0s 393us/step
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 387us/step


In [ ]:
train_y = features.train_y
val_y = features.val_y

display(train_y.shape)
display(val_y.shape)

(10981,)

'melspectrogram'

(10981, 60)

'mfcc'

(10981, 20)

'chroma'

(10981, 12)

'rms'

(10981, 1)

In [ ]:
# Avgpooled Features
train_features_1D = features.train_features_1D
for key in train_features_1D.keys():
  display(key)
  display(train_features_1D[key].shape)

In [ ]:
# Avgpooled Features
val_features_1D = features.val_features_1D
for key in val_features_1D.keys():
  display(key)
  display(val_features_1D[key].shape)

In [ ]:
# Not avgpooled Features
train_features_2D = features.train_features_2D
for key in train_features_2D.keys():
  display(key)
  display(train_features_2D[key].shape)

In [ ]:
# Not avgpooled Features
val_features_2D = features.val_features_2D
for key in val_features_2D.keys():
  display(key)
  display(val_features_2D[key].shape)

In [ ]:
train_ids = features.train_ids
val_ids = features.val_ids

### Encode Classes

In [ ]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [1 1 1 ... 5 5 5]
Encoded validation labels: [1 1 1 ... 5 5 5]


In [ ]:
# Avg Pooled

train_features_1D['label'] = train_y_encoded
val_features_1D['label'] = val_y_encoded

train_features_1D['id'] = train_ids
val_features_1D['id'] = val_ids

# Not Avg Pooled

train_features_2D['label'] = train_y_encoded
val_features_2D['label'] = val_y_encoded

train_features_2D['id'] = train_ids
val_features_2D['id'] = val_ids

In [ ]:
merged_dict_1D = {'train': train_features_1D, 'val': val_features_1D}
merged_dict_1D

In [ ]:
merged_dict_2D = {'train': train_features_2D, 'val': val_features_2D}
merged_dict_2D

{'train': {'melspectrogram': array([[1.7294491e-05, 1.8239583e-05, 1.9485902e-05, ..., 5.1940111e-07,
          3.2287385e-08, 2.7940120e-08],
         [7.9314253e-07, 8.0714307e-07, 8.3096978e-07, ..., 1.3027967e-06,
          8.4759665e-07, 8.0172282e-07],
         [3.0612518e-06, 3.0607123e-06, 3.0592107e-06, ..., 6.0072955e-07,
          3.6999687e-08, 1.4126979e-08],
         ...,
         [1.0815166e-06, 1.0910663e-06, 1.1071655e-06, ..., 4.2367597e-07,
          1.3173792e-07, 1.2475495e-07],
         [2.1907644e-07, 2.7223422e-07, 3.6665881e-07, ..., 5.7446897e-07,
          1.8625977e-07, 1.7997159e-07],
         [2.4198832e-06, 2.6966893e-06, 3.1864115e-06, ..., 1.2659899e-05,
          2.3157411e-07, 2.2612821e-07]], dtype=float32),
  'mfcc': array([[-1.0000000e+00, -2.3626368e-01, -5.0428110e-01, ...,
           5.1737577e-03,  1.1148715e-02,  3.4610374e-04],
         [-1.0000000e+00, -2.4848318e-01, -4.7058651e-01, ...,
           2.9826174e-03,  9.7121680e-03, -5.3395964e

### Save the merged dictionary to a pkl

In [ ]:
# Avrg Pooled
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/Annotated/Regular/AveragePooled/split_features_1s_all_1D.pkl', 'wb') as file:
  pickle.dump(merged_dict_1D, file)
del file

In [ ]:
# Avrg Pooled
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/Annotated/Regular/NotAveragePooled/split_features_1s_all_2D.pkl', 'wb') as file:
  pickle.dump(merged_dict_2D, file)
del file